**Imports**

In [21]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

# Feature Selection
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Regression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import LassoLars
from sklearn.neural_network import MLPRegressor

import matplotlib.mlab as mlab

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

**Loading Data**

In [22]:

# Load training data
X = np.genfromtxt('training.csv', 
                  delimiter=',', 
                  dtype=None,
                  skip_header = 1,
                  usecols=range(1, 3594)) # Load columns 0 to 3594 inclusive

Ca = np.genfromtxt('training.csv', 
                   delimiter=',', 
                   dtype=None,
                   skip_header = 1,
                   usecols=3595) # Load Mehlich-3 extractable Calcium data

P = np.genfromtxt('training.csv', 
                   delimiter=',', 
                   dtype=None,
                   skip_header = 1,
                   usecols=3596) # Load Mehlich-3 extractable Phosphorus data

pH = np.genfromtxt('training.csv', 
                   delimiter=',', 
                   dtype=None,
                   skip_header = 1,
                   usecols=3597) # Load pH data

SOC = np.genfromtxt('training.csv', 
                    delimiter=',', 
                    dtype=None,
                    skip_header = 1,
                    usecols=3598) # Load Soil Organic Carbon data

Sand = np.genfromtxt('training.csv', 
                     delimiter=',', 
                     dtype=None,
                     skip_header = 1,
                     usecols=3599) # Load Sand Content data

# Each record in the training data loads as a tuple.
# Convert it to a 2D array
# TODO:  There's a better way to do this somewhere.
new_X = []
for i in range(X.shape[0]):
    new_X.append(list(X[i]))
X = np.array(new_X)

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Ca, P, pH, SOC, Sand = X[shuffle], Ca[shuffle], P[shuffle], pH[shuffle], SOC[shuffle], Sand[shuffle] 

# Define the size of the evaluation data set
evalSetSize = 100

# Define the size of the dev data set
devSetSize = 100

# Total size to hold out
holdOutSize = evalSetSize + devSetSize

eval_data = X[0:evalSetSize]
eval_Ca_labels = Ca[0:evalSetSize]
eval_P_labels = P[0:evalSetSize]
eval_pH_labels = pH[0:evalSetSize]
eval_SOC_labels = SOC[0:evalSetSize]
eval_Sand_labels = Sand[0:evalSetSize]

dev_data = X[evalSetSize:holdOutSize]
dev_Ca_labels = Ca[evalSetSize:holdOutSize]
dev_P_labels = P[evalSetSize:holdOutSize]
dev_pH_labels = pH[evalSetSize:holdOutSize]
dev_SOC_labels = SOC[evalSetSize:holdOutSize]
dev_Sand_labels = Sand[evalSetSize:holdOutSize]

dev_labels = [dev_Ca_labels, dev_P_labels, dev_pH_labels, dev_SOC_labels, dev_Sand_labels]
eval_labels = [eval_Ca_labels, eval_P_labels, eval_pH_labels, eval_SOC_labels, eval_Sand_labels]

outcome_vars = ['Ca', 'P', 'pH', 'Soc', 'Sand']

train_data = X[holdOutSize:]
train_Ca_labels = Ca[holdOutSize:]
train_P_labels = P[holdOutSize:]
train_pH_labels = pH[holdOutSize:]
train_SOC_labels = SOC[holdOutSize:]
train_Sand_labels = Sand[holdOutSize:]
train_labels = [train_Ca_labels, train_P_labels, train_pH_labels, train_SOC_labels, train_Sand_labels]

print(eval_Ca_labels.shape)
print(dev_Ca_labels.shape)
print(train_Ca_labels.shape)


print('Number of features:', dev_data.shape[1])
print('Number of training examples:', train_data.shape[0])
print('Number of dev examples:', dev_data.shape[0])
print('Number of eval examples:', eval_data.shape[0])

#TODO: Dev / Train data have 2 string columns, the first and the last. 
#The first is an ID, and the last is a categorical with 2 possible values apparently. 
#Encode / transform these so they are useful

# Load test data
test_x = np.genfromtxt('sorted_test.csv', 
                                delimiter=',', 
                                dtype=None,
                                skip_header = 1,
                                usecols=range(1, 3594)) # Load columns 0 to 3594 inclusive

test_ids = np.genfromtxt('sorted_test.csv', 
                                delimiter=',', 
                                dtype=None,
                                skip_header = 1,
                                usecols=0) # Load columns 0 to 3594 inclusive

new_test_x = []
for i in range(test_x.shape[0]):
    new_test_x.append(list(test_x[i]))
test_x = np.array(new_test_x)


print('Number of test examples:', test_x.shape[0])




(100L,)
(100L,)
(957L,)
('Number of features:', 3593L)
('Number of training examples:', 957L)
('Number of dev examples:', 100L)
('Number of eval examples:', 100L)
('Number of test examples:', 727L)


Experiment on PCA: Analyze the explaned variances for PCA over our features. We observe that the first 20 components explain increasing portions of the variance, however after 20 components, the subsequent ones don't really help.

In [23]:
    # Fit PCA for the train data
    pca = KernelPCA(n_components=200,kernel='rbf')
    pca.fit(train_data)
    
    accum = 0.0
    # Display the fraction of the variance explained by the top 50 components
    for i in range(1, 200):
        print('k: ' + str(i) + ', Explained variance ratio: ' + str(pca.lambdas_[i]/sum(pca.lambdas_)))
        accum = accum + pca.lambdas_[i]/sum(pca.lambdas_)
        print('k: ' + str(i) + ', Cumulative variance ratio: ' + str(accum))
        #accum = accum + pca.explained_variance_ratio_[i]
        #print('k: ' + str(i) + ', Variance fraction explained: ' + 
        #      str(pca.explained_variance_ratio_[i]) + ', accum variance explained: ' + str(accum))
        
    

k: 1, Explained variance ratio: 0.0907824042451
k: 1, Cumulative variance ratio: 0.0907824042451
k: 2, Explained variance ratio: 0.0626229902585
k: 2, Cumulative variance ratio: 0.153405394504
k: 3, Explained variance ratio: 0.0385819230313
k: 3, Cumulative variance ratio: 0.191987317535
k: 4, Explained variance ratio: 0.0295134335574
k: 4, Cumulative variance ratio: 0.221500751092
k: 5, Explained variance ratio: 0.0199618529456
k: 5, Cumulative variance ratio: 0.241462604038
k: 6, Explained variance ratio: 0.0177146062861
k: 6, Cumulative variance ratio: 0.259177210324
k: 7, Explained variance ratio: 0.0156011617927
k: 7, Cumulative variance ratio: 0.274778372117
k: 8, Explained variance ratio: 0.00992179021016
k: 8, Cumulative variance ratio: 0.284700162327
k: 9, Explained variance ratio: 0.0062956883535
k: 9, Cumulative variance ratio: 0.29099585068
k: 10, Explained variance ratio: 0.00550822369675
k: 10, Cumulative variance ratio: 0.296504074377
k: 11, Explained variance ratio: 0.0

**Feature selectors**

In [36]:

def getFeatureSelectors():
     return [
        #['lasso', SelectFromModel(LassoCV()) ], # Doesn't work
        #['linearc0.01', SelectFromModel(LinearSVC(C=0.01, penalty="l1")) ],
        #['linearc0.1', SelectFromModel(LinearSVC(C=0.1, penalty="l1")) ],
        #['linearc11', SelectFromModel(LinearSVC(C=1, penalty="l1")) ],
        #['kbest100', SelectKBest(k=100)],
        #['kbest250', SelectKBest(k=250)],
        #['pca5', PCA(n_components=5)],
        #['pca10', PCA(n_components=10)],
        ['pca20', PCA(n_components=20)],
        ['pca30', PCA(n_components=30)],
        ['pca30rbf', KernelPCA(n_components=30,kernel='rbf')],
        ['pca20kbest5', FeatureUnion([("pca5", PCA(n_components=20)), ("kbest5", SelectKBest(k=5))])],
        ['pca20kbest50', FeatureUnion([("pca5", PCA(n_components=20)), ("kbest50", SelectKBest(k=50))])],
        ['pca20kbest250', FeatureUnion([("pca5", PCA(n_components=20)), ("kbest250", SelectKBest(k=250))])],
     ]


**Classifiers**

In [37]:
# Predict the mean value of an array
def PredictMean(labels):
    mean = np.mean(labels)
    mean_array = mean*np.ones(labels.shape)
    return mean_array

In [38]:
# Get the means of the dev data for our reference

Ca_mean = PredictMean(dev_Ca_labels)
print('Calcium Mean: ', Ca_mean[0])

P_mean = PredictMean(dev_P_labels)
print('Phosphorus Mean: ', P_mean[0])

pH_mean = PredictMean(dev_pH_labels)
print('pH Mean: ', pH_mean[0])

SOC_mean = PredictMean(dev_SOC_labels)
print('SOC Mean: ', SOC_mean[0])

Sand_mean = PredictMean(dev_Sand_labels)
print('Sand Mean: ', Sand_mean[0])

('Calcium Mean: ', 0.060906193376832396)
('Phosphorus Mean: ', 0.042220159248226287)
('pH Mean: ', 0.027768020593356191)
('SOC Mean: ', 0.021567274983507304)
('Sand Mean: ', 0.058996238160944625)


In [39]:



def getClassifiers():
     return [
        ['KNN', KNeighborsRegressor(), {'n_neighbors':[1, 2, 3, 5, 8]}],
        ['SVRdict', SVR(cache_size=200), {'C':[0.1,1.0,100.0,1000.0],'kernel':['linear','rbf']}],
        #['SVR', SVR(cache_size=200, kernel='linear', C=1.0, epsilon=0.05, shrinking=False),{}]
        ['Lasso', Lasso(), {'alpha':[0.01, 0.05, 0.25, 0.9]}],
        #['LassoLars', LassoLars(), {'alpha':[0.01, 0.1, 0.5, 1.0]}],            
        ['RandomForest', RandomForestRegressor(), {'n_estimators':[1, 2, 3, 5, 8]}],
        ['nn', MLPRegressor(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2, 2)), {}]
        #['RandomForest', RandomForestRegressor(), {'n_estimators':[1, 2, 3, 5, 8]}]
    ]


**Test combinations of selectors and classifiers**

In [40]:


# For each outcome variable, for each classifier and for each selector, it will obtain and print 
# the best hyperparameters and inmediately print the mean squared error.
# Finally, after finishing the calculations, it will print the methods and score ordered by score (MSE)
# To see what would be the final score we'd get, you can get the best MSE score for each outcome variable,
# and calculate the average of them.
def run():
    
    allResults = []
    
    scaler = Normalizer().fit(train_data)

    
    transformedTrainData = scaler.transform(train_data)
    transformedDevData = scaler.transform(dev_data)

    
    for outcomeVarIndex in range(0, 5):
        print('*************************************************************')
        print('Outcome Variable:', outcome_vars[outcomeVarIndex])
        print('*************************************************************')

        results = []
            
        scaler = Normalizer().fit(train_data)
        
        # Get the mean value of the outcome variable
        DevMean = PredictMean(dev_labels[outcomeVarIndex])[0]

        for selector in getFeatureSelectors():
            
            selectedTrainData = selector[1].fit(transformedTrainData, train_labels[outcomeVarIndex]).transform(transformedTrainData)
            selectedDevData = selector[1].transform(transformedDevData)
                
            for classifier in getClassifiers():

                print('-------------------------------------------------------')
                print(selector[0] + ' ' + classifier[0])

                grid_search = GridSearchCV(classifier[1], param_grid=classifier[2],cv=5)

                grid_search.fit(selectedTrainData, train_labels[outcomeVarIndex])
                print(grid_search.best_estimator_)

                # Mean Squared Error:  (y_true - y_pred)**2.sum()
                meanSquaredError = 0.0
                for i in range(len(selectedDevData)):
                    diff = grid_search.predict(selectedDevData[i].reshape(1, -1)) - dev_labels[outcomeVarIndex][i]
                    squaredDiff = diff ** 2
                    meanSquaredError = meanSquaredError + squaredDiff
                    
                #meanSquaredError = meanSquaredError / float(len(selectedDevData))
                
                # Residual Sum of squares:  (y_true - y_mean)**2.sum()
                residualSquaredError = 0.0
                for i in range(len(selectedDevData)):
                    diff = dev_labels[outcomeVarIndex][i] - DevMean
                    squaredDiff = diff ** 2
                    residualSquaredError = residualSquaredError + squaredDiff     
                    
                myScore = 1 - meanSquaredError/residualSquaredError
                
                print('Mean Squared Error: ', str(meanSquaredError / float(len(selectedDevData))))
                print('Residual Squared Error: ', str(residualSquaredError))
                print('Calculated Score: ', str(1 - meanSquaredError/residualSquaredError))
                print('Score: ' + str(grid_search.score(selectedDevData, dev_labels[outcomeVarIndex])))
                
                # Store in an array, for each combination, the following:
                # [selector name, classifier name, mean squared error, selector instance, classifier instance]
                results.append([selector[0], classifier[0], myScore, selector[1], grid_search])

                                                            
        sortedResults = sorted(results, key=lambda result: result[2], reverse=True)
        for result in sortedResults:
            print('Selector: ' + str(result[0]) + ', Classifier: ' + str(result[1])  + ', Score: ' + str(result[2]))
        
        copyResults = sortedResults[:]
        allResults.append(copyResults)
        
    #TODO Calculate columnwise mean of the mean squared error
    #Each item has the best result on item 0
    squaredErrorSum = 0.0
    
    bestModels = []
    print('-------------------------------------------------------')
    print('Best Results')
    print('-------------------------------------------------------')
    for i in range(len(allResults)):

        columnResults = allResults[i]
        bestColumnResult = columnResults[0]
        squaredError = bestColumnResult[2]
        print('Outcome Variable: ' + outcome_vars[i] + ', Selector: ' + str(bestColumnResult[0]) + 
              ', Classifier: ' + str(bestColumnResult[1])  + ', Score: ' + str(bestColumnResult[2]))
        squaredErrorSum = squaredErrorSum + squaredError
        bestModels.append(bestColumnResult)
        
        
    print('Best result obtained: ' + str(squaredErrorSum / 5.0))
    
    return bestModels

models = run()


*************************************************************
('Outcome Variable:', 'Ca')
*************************************************************
-------------------------------------------------------
pca20 KNN
KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')
('Mean Squared Error: ', '[ 0.20391923]')
('Residual Squared Error: ', '126.489297838')
('Calculated Score: ', '[ 0.83878539]')
Score: 0.838785387535
-------------------------------------------------------
pca20 SVRdict
SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
('Mean Squared Error: ', '[ 0.31648473]')
('Residual Squared Error: ', '126.489297838')
('Calculated Score: ', '[ 0.74979327]')
Score: 0.749793272792
-------------------------------------------------------
pca20 Lasso
Lasso(alpha=0.01, copy_X=True, 

**Predictions based on our dev data**


Test on the evaluation data

In [41]:
myScore = []

scaler = Normalizer().fit(train_data)
transformedEvalData = scaler.transform(eval_data)
#transformedEvalData = eval_data

# Use the appropriate model to estimate the 5 outcome variables
for outcomeVarIndex in range(0, 5):
        
    # Grab selector and classifier
    selector = models[outcomeVarIndex][3]
    classifier = models[outcomeVarIndex][4]
        
    # Transform the input variables
    selectedSample = selector.transform(transformedEvalData)
        
    # Predict
    myScore.append(classifier.score(selectedSample,eval_labels[outcomeVarIndex]))
        
print(myScore)    

[0.71843805143027306, 0.34060013203988482, 0.71094918313111499, 0.93328557538590817, 0.85979394860377367]


In [42]:
scaler = Normalizer().fit(train_data)
normalizedTestData = scaler.transform(test_x)
#normalizedTestData = test_x

allPredictions = []

# Iterate through test samples
for sampleIndex in range(len(test_x)):
    
    sampleId = test_ids[sampleIndex]
    sample = normalizedTestData[sampleIndex]
    
    currentSamplePredictions = []
    
    # Use the appropriate model to estimate the 5 outcome variables
    for outcomeVarIndex in range(0, 5):
        
        # Grab selector and classifier
        selector = models[outcomeVarIndex][3]
        classifier = models[outcomeVarIndex][4]
        
        # Transform the input variables
        selectedSample = selector.transform(sample.reshape(1, -1))
        
        # Predict
        predicted = classifier.predict(selectedSample.reshape(1, -1))
        
        # Store
        currentSamplePredictions.append(predicted[0])
    
    allPredictions.append(currentSamplePredictions)
    

print(allPredictions)
       

[[-0.45182920536665244, -0.30187414596763368, -0.5741683950247044, -0.49351730715911923, 0.76355530824945239], [0.3643799009749788, 0.53853307897997338, 0.95143173799029868, -0.49351730715911923, -0.49621701348399633], [-0.11421517569695498, -0.19696734559694482, 0.0077543206064288467, -0.35233719088969484, -0.89964863313277432], [-0.41208608706934574, -0.28919310416458366, -0.49028535800237844, 1.1642803815032456, -0.19427762519340139], [-0.42897835860182515, -0.31686083173487506, -1.4519070301287385, -0.35528970994898712, -0.040770625264149318], [-0.40404490301792545, -0.3295418735379253, -0.61288000072671112, -0.49351730715911923, 0.15967653166826135], [0.11018592455814362, -0.18198065982970327, -0.45340748916848556, 0.93892277955805903, -1.2447222197505967], [-0.29065842286840421, -0.28227617227201068, 0.81117318293018237, -0.49351730715911923, -0.10166596407905933], [-0.086562758599265849, 1.4527209107816941, 0.57631493385521393, -0.046062714757278744, -0.53681390602727053], [-0.2

**Generate CSV file for kaggle submission**

In [44]:
# Print header
header = 'PIDN,Ca,P,pH,SOC,Sand'
#np.savetxt('test.out', header, delimiter=',')  

filename = 'test6.txt'
# Clean file
open(filename, 'w').close()
with open(filename, 'w') as f:
    f.write('PIDN,Ca,P,pH,SOC,Sand\n')  # python will convert \n to os.linesep

    # Iterate through test samples
    for i in range(len(allPredictions)):

        pred = allPredictions[i]
        testId = test_ids[i]
        text = testId + ',' + str(pred[0]) + ',' + str(pred[1]) + ',' + str(pred[2]) + ',' + str(pred[3]) + ',' + str(pred[4]) + '\n'
        f.write(text) 
    
f.close()

Investigation: Sand variable

In [ ]:


#!/usr/bin/env python



# the histogram of the data
n, bins, patches = plt.hist(train_Sand_labels, 50, normed=1, facecolor='green', alpha=0.75)

plt.title(r'$\mathrm{Sand:}$')
#plt.axis([40, 160, 0, 0.03])
plt.grid(True)

plt.show()

scaler = Normalizer().fit(train_data)

transformedTrainData = scaler.transform(train_data)
transformedDevData = scaler.transform(dev_data)

selector = FeatureUnion([("pca5", PCA(n_components=20)), ("kbest5", SelectKBest(k=250))]).fit(transformedTrainData, train_Sand_labels)
#selector = SelectFromModel(LassoCV(), threshold=0.25).fit(transformedTrainData, train_Sand_labels)
selectedTrainData = selector.transform(transformedTrainData)
selectedDevData = selector.transform(transformedDevData)


classifier =  LassoLars(alpha=0.01)#LassoLars is doing very well on dev set, but horribly wrong in test set. bug or really overfitted?
classifier.fit(transformedTrainData, train_Sand_labels)

print classifier.score(transformedDevData, dev_Sand_labels)

meanSquaredError = 0.0
for i in range(len(transformedDevData)):
    diff = classifier.predict(transformedDevData[i].reshape(1, -1)) - dev_Sand_labels[i]
    #print str(classifier.predict(transformedDevData[i].reshape(1, -1))) + ', ' + str(dev_Sand_labels[i])
    squaredDiff = diff ** 2
    meanSquaredError = meanSquaredError + squaredDiff
    

    
print meanSquaredError / float(len(transformedDevData))